# 利用tf-idf构建词向量

In [185]:
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  
import pandas as pd
import numpy as np
fr = pd.read_csv('C:\\Users\\王教授\Desktop\\大数据挑战赛预选赛\\data\\remake_train.csv',engine = 'python')
fr_ = pd.read_csv('C:\\Users\\王教授\Desktop\\大数据挑战赛预选赛\\data\\20190513_test.csv',engine = 'python')
corpus = []
Labels = []
for i in fr['review']:
    corpus.append(i)
for i in fr_['review']:
    corpus.append(i)
# 1、构建词频矩阵，将文本中的词语转换成词频矩阵
vectorizer=CountVectorizer()
# 2、统计每个词的tf-idf权值
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus))  
# 3、获取词袋模型中的关键词（默认去掉停用词）
word = vectorizer.get_feature_names()
# 4、获取tf-idf矩阵，a[i][j]表示j词在i篇文本中的tf-idf权重
weight = tfidf.toarray()  # 每个关键词在相应句子中的tf-idf值
dic = vectorizer.vocabulary_    # 得到每个关键词对应的索引
X = weight[0:6328]
XTest = weight[6328:]
Y = fr['label']

In [186]:
Y.replace("Positive", 1, inplace=True)
Y.replace("Negative", 0, inplace=True)

# 使用pca对词向量进行降维处理（提高泛化能力，防止过拟合，提高运行效率）

In [117]:
from sklearn.decomposition import PCA
pca = PCA(100)  # 降维
pca_data = pca.fit_transform(X)

In [157]:
(pca_data+np.abs(pca_data))/2

array([[0.01161966, 0.        , 0.04167007, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00039089, 0.02062857,
        0.02323341],
       [0.        , 0.        , 0.01917368, ..., 0.0580126 , 0.02121947,
        0.04318964],
       ...,
       [0.        , 0.        , 0.        , ..., 0.02485908, 0.        ,
        0.        ],
       [0.13507671, 0.02678847, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00280748, ..., 0.00160068, 0.01067999,
        0.        ]])

# 使用多项式贝叶斯模型进行分类训练

In [187]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)   # alpha是拉普拉斯平滑系数
clf.fit(X, Y)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [177]:
X.shape,Y.shape   

((6328, 22238), (6328,))

# 使预测结果以概率的形式呈现

In [169]:
resultTest = clf.predict_proba(XTest)[:,1]
id = [i for i in range(1,len(resultTest)+1)]
len(resultTest),len(id)

(2712, 2712)

# 将预测的结果写入csv文件中

In [179]:
dataframe = pd.DataFrame({'ID':id,'Pred':resultTest})
dataframe.to_csv("C:\\Users\\王教授\\Desktop\\大数据挑战赛预选赛\\data\\test.csv",index=False,sep=',')

# k折交叉验证 

In [188]:
from sklearn.model_selection import cross_val_score
# k折交叉验证 计算AUC
scores = cross_val_score(clf, X, Y, scoring="roc_auc", cv=5)
scores.mean()

0.8567583698797513

# 调用逻辑回归模型作为分类器

In [125]:
from sklearn.linear_model import LogisticRegression
# 逻辑回归
logreg = LogisticRegression(C=1e5,penalty='l2')
logreg.fit(X, Y)   # 训练
r = cross_val_score(logreg, X, Y, scoring="roc_auc", cv=5)
r.mean()

0.8181775423238244

In [91]:
logreg.fit(pca_data, Y)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [65]:
from sklearn.naive_bayes import GaussianNB

Guss = GaussianNB().fit(pca_data, Y)
gc = cross_val_score(Guss, pca_data, Y, scoring="accuracy", cv=5)
gc.mean()

0.6342293810817361

In [162]:

from sklearn.naive_bayes import BernoulliNB

bnl = BernoulliNB()

bnl.fit(X, Y)
bnlc = cross_val_score(Guss, X, Y, scoring="roc_auc", cv=5)
bnlc.mean()

0.710936412324356